# MAP Competition - Advanced Baseline v2.0

## Overview
Robust implementation of improvement strategies:
- **Strategy 2**: Mathematical Feature Engineering
- **Strategy 4**: MAP@3 Optimization

**Target**: Beat current #1 position (Public LB: 0.841)  
**Expected improvement**: +0.025-0.065 MAP@3 score

## Key Improvements
- Robust error handling for all feature extraction
- Safe mathematical feature computation
- LightGBM instead of XGBoost for stability
- Comprehensive text preprocessing
- MAP@3 optimized prediction generation

## 1. Setup and Imports

In [1]:
%%time

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import lightgbm as lgb
from scipy import sparse

# Text Processing
try:
    import nltk
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    print("NLTK WordNetLemmatizer loaded successfully")
except:
    lemmatizer = None
    print("NLTK WordNetLemmatizer not available, using basic processing")

import time
import os

print("All imports completed successfully")

NLTK WordNetLemmatizer loaded successfully
All imports completed successfully
CPU times: user 2.4 s, sys: 403 ms, total: 2.81 s
Wall time: 1.28 s


## 2. Data Loading

In [2]:
%%time

# Data paths - Local development
train_path = "/Users/osawa/kaggle/map-charting-student-math-misunderstandings/data/raw/train.csv"
test_path = "/Users/osawa/kaggle/map-charting-student-math-misunderstandings/data/raw/test.csv"
sample_path = "/Users/osawa/kaggle/map-charting-student-math-misunderstandings/data/raw/sample_submission.csv"

# For Kaggle submission, uncomment these lines:
# train_path = "/kaggle/input/map-charting-student-math-misunderstandings/train.csv"
# test_path = "/kaggle/input/map-charting-student-math-misunderstandings/test.csv"
# sample_path = "/kaggle/input/map-charting-student-math-misunderstandings/sample_submission.csv"

print("Loading data...")
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_path)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

# Prepare targets
train['Misconception'] = train['Misconception'].fillna('NA')
train['target_combined'] = train['Category'] + ':' + train['Misconception']

print(f"Unique categories: {train['Category'].nunique()}")
print(f"Unique misconceptions: {train['Misconception'].nunique()}")
print(f"Unique combinations: {train['target_combined'].nunique()}")

print("\nTarget distribution:")
print(train['Category'].value_counts())

Loading data...
Train shape: (36696, 7)
Test shape: (3, 5)
Unique categories: 6
Unique misconceptions: 36
Unique combinations: 65

Target distribution:
Category
True_Correct           14802
False_Misconception     9457
False_Neither           6542
True_Neither            5265
True_Misconception       403
False_Correct            227
Name: count, dtype: int64
CPU times: user 61.8 ms, sys: 13 ms, total: 74.8 ms
Wall time: 78.5 ms


## 3. Feature Extraction Classes

In [3]:
class MathematicalFeatureExtractor:
    """Extract mathematical features from text with robust error handling"""
    
    def __init__(self):
        # Mathematical patterns (fixed regex escaping)
        self.fraction_pattern = re.compile(r'\\frac\{([^}]+)\}\{([^}]+)\}')
        self.simple_fraction_pattern = re.compile(r'(\d+)\s*/\s*(\d+)')
        self.decimal_pattern = re.compile(r'\d+\.\d+')
        self.percentage_pattern = re.compile(r'\d+%')
        self.number_pattern = re.compile(r'\b\d+\b')
        self.operation_pattern = re.compile(r'[+\-*/=]')
        
        # Mathematical concepts
        self.math_concepts = {
            'fraction': ['fraction', 'numerator', 'denominator', 'over', 'divided'],
            'decimal': ['decimal', 'point', 'place', 'tenths', 'hundredths'],
            'percentage': ['percent', 'percentage', '%', 'out of 100'],
            'addition': ['add', 'plus', 'sum', 'total', 'altogether'],
            'subtraction': ['subtract', 'minus', 'difference', 'take away'],
            'multiplication': ['multiply', 'times', 'product', 'of'],
            'division': ['divide', 'quotient', 'split', 'share'],
            'comparison': ['greater', 'less', 'equal', 'bigger', 'smaller', 'same']
        }
        
    def safe_extract_numbers(self, text):
        """Safely extract numbers with bounds checking"""
        try:
            numbers = []
            for match in self.number_pattern.findall(str(text)):
                try:
                    num = float(match)
                    if 0 <= num <= 1e6:  # Reasonable bounds
                        numbers.append(num)
                except (ValueError, OverflowError):
                    continue
            return numbers
        except:
            return []
    
    def extract_numerical_features(self, text):
        """Extract numerical features safely"""
        text = str(text)
        features = {}
        
        # Count patterns
        features['fraction_count'] = len(self.fraction_pattern.findall(text))
        features['simple_fraction_count'] = len(self.simple_fraction_pattern.findall(text))
        features['decimal_count'] = len(self.decimal_pattern.findall(text))
        features['percentage_count'] = len(self.percentage_pattern.findall(text))
        features['operation_count'] = len(self.operation_pattern.findall(text))
        
        # Number analysis
        numbers = self.safe_extract_numbers(text)
        features['number_count'] = len(numbers)
        
        if numbers:
            features['max_number'] = max(numbers)
            features['min_number'] = min(numbers)
            features['number_range'] = features['max_number'] - features['min_number']
            features['avg_number'] = np.mean(numbers)
        else:
            features['max_number'] = 0
            features['min_number'] = 0
            features['number_range'] = 0
            features['avg_number'] = 0
        
        return features
    
    def extract_concept_features(self, text):
        """Extract mathematical concept features"""
        text_lower = str(text).lower()
        features = {}
        
        for concept, keywords in self.math_concepts.items():
            count = sum(1 for kw in keywords if kw in text_lower)
            features[f'{concept}_concept'] = min(count, 10)  # Cap at 10
            features[f'has_{concept}'] = 1 if count > 0 else 0
            
        return features
    
    def extract_complexity_features(self, text):
        """Extract text complexity features"""
        text = str(text)
        features = {}
        
        # Basic text stats
        features['text_length'] = min(len(text), 5000)
        words = text.split()
        features['word_count'] = len(words)
        
        if words:
            features['avg_word_length'] = min(np.mean([len(w) for w in words]), 20)
        else:
            features['avg_word_length'] = 0
            
        features['sentence_count'] = min(len(re.split(r'[.!?]', text)), 50)
        features['has_latex'] = 1 if '\\' in text else 0
        features['parentheses_count'] = min(text.count('(') + text.count(')'), 20)
        
        return features
    
    def extract_all_features(self, text):
        """Extract all features with error handling"""
        all_features = {}
        
        try:
            all_features.update(self.extract_numerical_features(text))
        except Exception:
            pass
            
        try:
            all_features.update(self.extract_concept_features(text))
        except Exception:
            pass
            
        try:
            all_features.update(self.extract_complexity_features(text))
        except Exception:
            pass
        
        # Ensure all values are finite
        for key, value in all_features.items():
            if not np.isfinite(value):
                all_features[key] = 0
                
        return all_features

print("MathematicalFeatureExtractor defined")

MathematicalFeatureExtractor defined


In [4]:
class SemanticFeatureExtractor:
    """Extract semantic relationship features"""
    
    @staticmethod
    def word_overlap_similarity(text1, text2):
        """Calculate word overlap similarity"""
        try:
            words1 = set(str(text1).lower().split())
            words2 = set(str(text2).lower().split())
            if not words1 or not words2:
                return 0
            intersection = len(words1.intersection(words2))
            union = len(words1.union(words2))
            return intersection / union if union > 0 else 0
        except:
            return 0
    
    def extract_features(self, df):
        """Extract semantic features from dataframe"""
        features = {}
        
        # Similarity features
        features['question_answer_similarity'] = df.apply(
            lambda x: self.word_overlap_similarity(x['QuestionText'], x['MC_Answer']), axis=1
        )
        
        features['question_explanation_similarity'] = df.apply(
            lambda x: self.word_overlap_similarity(x['QuestionText'], x['StudentExplanation']), axis=1
        )
        
        features['answer_explanation_similarity'] = df.apply(
            lambda x: self.word_overlap_similarity(x['MC_Answer'], x['StudentExplanation']), axis=1
        )
        
        # Length features
        q_len = df['QuestionText'].str.len().fillna(1)
        e_len = df['StudentExplanation'].str.len().fillna(0)
        
        features['explanation_question_length_ratio'] = np.clip(e_len / q_len, 0, 10)
        features['explanation_length'] = np.clip(e_len, 0, 1000)
        features['question_length'] = np.clip(q_len, 0, 1000)
        
        return pd.DataFrame(features)

print("SemanticFeatureExtractor defined")

SemanticFeatureExtractor defined


In [5]:
class TextProcessor:
    """Handle text preprocessing"""
    
    def __init__(self):
        self.lemmatizer = lemmatizer
    
    def clean_text(self, text):
        """Clean text for processing"""
        text = str(text)
        
        # Basic cleaning
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip().lower()
        
        # Keep alphanumeric, basic math, and spaces
        text = re.sub(r'[^a-zA-Z0-9\s+\-*/=().]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text
    
    def lemmatize_text(self, text):
        """Lemmatize text if lemmatizer available"""
        if self.lemmatizer:
            try:
                words = text.split()
                lemmatized = [self.lemmatizer.lemmatize(word) for word in words]
                return ' '.join(lemmatized)
            except:
                return text
        return text
    
    def process_text(self, text):
        """Full text processing pipeline"""
        text = self.clean_text(text)
        text = self.lemmatize_text(text)
        return text

print("TextProcessor defined")

TextProcessor defined


In [6]:
class MAP3Optimizer:
    """MAP@3 specific optimization and evaluation"""
    
    @staticmethod
    def map3_score(y_true, y_pred_proba, class_names):
        """Calculate MAP@3 score"""
        scores = []
        
        for i, true_label in enumerate(y_true):
            # Get top 3 predictions
            top_3_indices = np.argsort(y_pred_proba[i])[::-1][:3]
            
            # Find rank of true label
            score = 0.0
            for rank, pred_idx in enumerate(top_3_indices, 1):
                pred_label = class_names[pred_idx]
                if pred_label == true_label:
                    score = 1.0 / rank
                    break
            
            scores.append(score)
        
        return np.mean(scores)
    
    @staticmethod
    def generate_combined_predictions(cat_probs, misc_probs, category_classes, misconception_classes, top_k=3):
        """Generate combined Category:Misconception predictions"""
        predictions = []
        
        for i in range(len(cat_probs)):
            pred_combos = []
            
            # Get top categories and misconceptions
            top_cats = np.argsort(cat_probs[i])[::-1][:top_k]
            top_miscs = np.argsort(misc_probs[i])[::-1][:top_k]
            
            # Generate combinations
            for cat_idx in top_cats:
                cat_name = category_classes[cat_idx]
                cat_prob = cat_probs[i][cat_idx]
                
                if 'Misconception' in cat_name:
                    # Add top misconceptions for misconception categories
                    for misc_idx in top_miscs:
                        misc_name = misconception_classes[misc_idx]
                        misc_prob = misc_probs[i][misc_idx]
                        
                        if misc_name != 'NA':
                            combined_label = f"{cat_name}:{misc_name}"
                            combined_prob = cat_prob * misc_prob
                            pred_combos.append((combined_label, combined_prob))
                else:
                    # Non-misconception categories always use NA
                    combined_label = f"{cat_name}:NA"
                    pred_combos.append((combined_label, cat_prob))
            
            # Sort by probability and take top 3
            pred_combos.sort(key=lambda x: x[1], reverse=True)
            top_3 = [combo[0] for combo in pred_combos[:3]]
            
            # Ensure exactly 3 predictions
            while len(top_3) < 3:
                top_3.append("True_Correct:NA")
            
            predictions.append(top_3)
        
        return predictions

print("MAP3Optimizer defined")

MAP3Optimizer defined


## 4. Text Processing and Feature Creation

In [7]:
%%time

print("Creating combined text...")
train['combined_text'] = ("Question: " + train['QuestionText'].astype(str) + 
                         " Answer: " + train['MC_Answer'].astype(str) + 
                         " Explanation: " + train['StudentExplanation'].astype(str))

test['combined_text'] = ("Question: " + test['QuestionText'].astype(str) + 
                        " Answer: " + test['MC_Answer'].astype(str) + 
                        " Explanation: " + test['StudentExplanation'].astype(str))

print("Processing text...")
processor = TextProcessor()
train['processed_text'] = train['combined_text'].apply(processor.process_text)
test['processed_text'] = test['combined_text'].apply(processor.process_text)

print("Text processing completed")
print(f"Sample processed text: {train['processed_text'].iloc[0][:200]}...")

Creating combined text...
Processing text...
Text processing completed
Sample processed text: question what fraction of the shape is not shaded give your answer in its simplest form. image a triangle split into 9 equal smaller triangles. 6 of them are shaded. answer ( frac 1 3 ) explanation 0n...
CPU times: user 3.86 s, sys: 1.12 s, total: 4.98 s
Wall time: 5.29 s


In [8]:
%%time

print("Extracting mathematical features...")
math_extractor = MathematicalFeatureExtractor()

train_math_features = []
for i, text in enumerate(train['combined_text']):
    if i % 5000 == 0:
        print(f"  Processing {i}/{len(train)}")
    features = math_extractor.extract_all_features(text)
    train_math_features.append(features)

test_math_features = []
for text in test['combined_text']:
    features = math_extractor.extract_all_features(text)
    test_math_features.append(features)

train_math_df = pd.DataFrame(train_math_features).fillna(0)
test_math_df = pd.DataFrame(test_math_features).fillna(0)

print(f"Mathematical features shape: {train_math_df.shape}")
print(f"Sample features: {list(train_math_df.columns)[:10]}")

Extracting mathematical features...
  Processing 0/36696
  Processing 5000/36696
  Processing 10000/36696
  Processing 15000/36696
  Processing 20000/36696
  Processing 25000/36696
  Processing 30000/36696
  Processing 35000/36696
Mathematical features shape: (36696, 32)
Sample features: ['fraction_count', 'simple_fraction_count', 'decimal_count', 'percentage_count', 'operation_count', 'number_count', 'max_number', 'min_number', 'number_range', 'avg_number']
CPU times: user 2.07 s, sys: 53.6 ms, total: 2.12 s
Wall time: 2.23 s


In [9]:
%%time

print("Extracting semantic features...")
semantic_extractor = SemanticFeatureExtractor()
train_semantic_df = semantic_extractor.extract_features(train)
test_semantic_df = semantic_extractor.extract_features(test)

print(f"Semantic features shape: {train_semantic_df.shape}")
print(f"Semantic features: {list(train_semantic_df.columns)}")

Extracting semantic features...
Semantic features shape: (36696, 6)
Semantic features: ['question_answer_similarity', 'question_explanation_similarity', 'answer_explanation_similarity', 'explanation_question_length_ratio', 'explanation_length', 'question_length']
CPU times: user 732 ms, sys: 14.5 ms, total: 747 ms
Wall time: 789 ms


In [10]:
%%time

print("Creating TF-IDF features...")
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 3),
    max_df=0.95,
    min_df=2,
    max_features=20000
)

all_text = pd.concat([train['processed_text'], test['processed_text']])
tfidf.fit(all_text)

train_tfidf = tfidf.transform(train['processed_text'])
test_tfidf = tfidf.transform(test['processed_text'])

print(f"TF-IDF shape: {train_tfidf.shape}")

Creating TF-IDF features...
TF-IDF shape: (36696, 20000)
CPU times: user 1.76 s, sys: 40.8 ms, total: 1.8 s
Wall time: 1.87 s


In [11]:
%%time

print("Combining features...")
train_math_sparse = sparse.csr_matrix(train_math_df.values)
test_math_sparse = sparse.csr_matrix(test_math_df.values)

train_semantic_sparse = sparse.csr_matrix(train_semantic_df.values)
test_semantic_sparse = sparse.csr_matrix(test_semantic_df.values)

train_features = sparse.hstack([train_tfidf, train_math_sparse, train_semantic_sparse])
test_features = sparse.hstack([test_tfidf, test_math_sparse, test_semantic_sparse])

print(f"Combined features shape: {train_features.shape}")
print(f"Feature breakdown:")
print(f"  TF-IDF: {train_tfidf.shape[1]}")
print(f"  Mathematical: {train_math_df.shape[1]}")
print(f"  Semantic: {train_semantic_df.shape[1]}")
print(f"  Total: {train_features.shape[1]}")

Combining features...
Combined features shape: (36696, 20038)
Feature breakdown:
  TF-IDF: 20000
  Mathematical: 32
  Semantic: 6
  Total: 20038
CPU times: user 17.6 ms, sys: 3.89 ms, total: 21.4 ms
Wall time: 20.5 ms


## 5. Model Training and Cross-Validation

In [12]:
# Prepare targets
categories = sorted(train['Category'].unique())
misconceptions = sorted(train['Misconception'].unique())

cat_to_idx = {cat: idx for idx, cat in enumerate(categories)}
misc_to_idx = {misc: idx for idx, misc in enumerate(misconceptions)}

train['cat_target'] = train['Category'].map(cat_to_idx)
train['misc_target'] = train['Misconception'].map(misc_to_idx)

print(f"Categories ({len(categories)}): {categories}")
print(f"Misconceptions ({len(misconceptions)}): {misconceptions[:10]}...")  # Show first 10

Categories (6): ['False_Correct', 'False_Misconception', 'False_Neither', 'True_Correct', 'True_Misconception', 'True_Neither']
Misconceptions (36): ['Adding_across', 'Adding_terms', 'Additive', 'Base_rate', 'Certainty', 'Definition', 'Denominator-only_change', 'Division', 'Duplication', 'Firstterm']...


In [ ]:
%%time

print("Starting cross-validation training...")
n_folds = 5  # Reduced for faster execution
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_cat_preds = np.zeros((len(train), len(categories)))
oof_misc_preds = np.zeros((len(train), len(misconceptions)))

test_cat_preds = np.zeros((len(test), len(categories)))
test_misc_preds = np.zeros((len(test), len(misconceptions)))

fold_map3_scores = []

def calculate_map3(true_labels, predictions):
    """Calculate MAP@3 score for validation"""
    scores = []
    for true_label, pred_list in zip(true_labels, predictions):
        score = 0.0
        for rank, pred in enumerate(pred_list, 1):
            if pred == true_label:
                score = 1.0 / rank
                break
        scores.append(score)
    return np.mean(scores)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_features, train['cat_target'])):
    print(f"\nFold {fold + 1}/{n_folds}")
    print(f"  Train size: {len(train_idx)}, Val size: {len(val_idx)}")
    
    # Split data
    X_train, X_val = train_features[train_idx], train_features[val_idx]
    y_cat_train, y_cat_val = train['cat_target'].iloc[train_idx], train['cat_target'].iloc[val_idx]
    y_misc_train, y_misc_val = train['misc_target'].iloc[train_idx], train['misc_target'].iloc[val_idx]
    
    # Train category model
    print("  Training category model...")
    cat_model = LogisticRegression(max_iter=1000, random_state=42, C=1.0)
    cat_model.fit(X_train, y_cat_train)
    
    # Train misconception model  
    print("  Training misconception model...")
    misc_model = lgb.LGBMClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        objective='multiclass',
        metric='multi_logloss',
        verbosity=-1
    )
    misc_model.fit(X_train, y_misc_train)
    
    # Predictions
    print("  Making predictions...")
    oof_cat_preds[val_idx] = cat_model.predict_proba(X_val)
    oof_misc_preds[val_idx] = misc_model.predict_proba(X_val)
    
    test_cat_preds += cat_model.predict_proba(test_features) / n_folds
    test_misc_preds += misc_model.predict_proba(test_features) / n_folds
    
    # Calculate MAP@3 for this fold (PROPER EVALUATION)
    print("  Calculating MAP@3...")
    val_predictions = MAP3Optimizer.generate_combined_predictions(
        oof_cat_preds[val_idx], oof_misc_preds[val_idx], 
        categories, misconceptions
    )
    
    val_true = train['target_combined'].iloc[val_idx].tolist()
    
    # Calculate proper MAP@3 score
    fold_map3 = calculate_map3(val_true, val_predictions)
    fold_map3_scores.append(fold_map3)
    
    # Also calculate top-1 accuracy for reference
    val_pred_first = [pred[0] for pred in val_predictions]
    fold_acc = np.mean([true == pred for true, pred in zip(val_true, val_pred_first)])
    
    print(f"  Fold {fold + 1} MAP@3: {fold_map3:.6f}")
    print(f"  Fold {fold + 1} Top-1 Acc: {fold_acc:.6f}")

print(f"\n🎯 CROSS-VALIDATION RESULTS (MAP@3):")
print(f"   Mean MAP@3: {np.mean(fold_map3_scores):.6f}")
print(f"   Std MAP@3:  {np.std(fold_map3_scores):.6f}")
print(f"   CV Stability: {np.std(fold_map3_scores)/np.mean(fold_map3_scores)*100:.2f}%")

cv_map3 = np.mean(fold_map3_scores)
print(f"\n📊 EXPECTED PERFORMANCE:")
print(f"   Public LB (estimated): {cv_map3:.6f}")
print(f"   vs Current #1 (0.841): {cv_map3 - 0.841:+.6f} ({'BEAT' if cv_map3 > 0.841 else 'MISS'})")
print(f"   vs Fork baseline (0.852): {cv_map3 - 0.852:+.6f} ({'BEAT' if cv_map3 > 0.852 else 'MISS'})")

print("Cross-validation completed successfully")

## 6. Final Evaluation and Predictions

In [ ]:
%%time

print("Calculating overall out-of-fold performance...")

# Generate OOF predictions using all CV predictions
oof_predictions = MAP3Optimizer.generate_combined_predictions(
    oof_cat_preds, oof_misc_preds, categories, misconceptions
)

# Calculate final MAP@3 on entire training set
true_labels = train['target_combined'].tolist()
overall_map3 = calculate_map3(true_labels, oof_predictions)

# Calculate accuracies for each position (for additional insights)
top1_acc = np.mean([true == pred[0] for true, pred in zip(true_labels, oof_predictions)])
top2_acc = np.mean([true in pred[:2] for true, pred in zip(true_labels, oof_predictions)])
top3_acc = np.mean([true in pred[:3] for true, pred in zip(true_labels, oof_predictions)])

print(f"\n📈 FINAL OUT-OF-FOLD PERFORMANCE:")
print(f"   MAP@3 Score: {overall_map3:.6f}")
print(f"   Top-1 Accuracy: {top1_acc:.6f}")
print(f"   Top-2 Accuracy: {top2_acc:.6f}")
print(f"   Top-3 Accuracy: {top3_acc:.6f}")

print(f"\n🎯 CONSISTENCY CHECK:")
print(f"   CV MAP@3: {cv_map3:.6f}")
print(f"   OOF MAP@3: {overall_map3:.6f}")
print(f"   Difference: {abs(cv_map3 - overall_map3):.6f} ({'GOOD' if abs(cv_map3 - overall_map3) < 0.005 else 'CHECK'})")

print(f"\n🏆 COMPETITION BENCHMARKS:")
improvement_vs_leader = overall_map3 - 0.841
improvement_vs_fork = overall_map3 - 0.852
print(f"   Current #1 (0.841): {improvement_vs_leader:+.6f} ({'✅ BEAT' if improvement_vs_leader > 0 else '❌ MISS'})")
print(f"   Fork baseline (0.852): {improvement_vs_fork:+.6f} ({'✅ BEAT' if improvement_vs_fork > 0 else '❌ MISS'})")

if overall_map3 > 0.841:
    print(f"\n🎉 SUCCESS! Expected to beat current #1 position!")
    if overall_map3 > 0.852:
        print(f"🚀 EXCELLENT! Expected to beat even the fork baseline!")
else:
    print(f"\n⚠️  Need improvement to beat current #1 position")
    print(f"   Gap to close: {0.841 - overall_map3:.6f}")

print(f"\n📊 VALIDATION SUMMARY:")
print(f"   Metric: MAP@3 (competition metric)")
print(f"   CV Method: {n_folds}-fold StratifiedKFold")
print(f"   Stability: {np.std(fold_map3_scores)/np.mean(fold_map3_scores)*100:.2f}% CV")
print(f"   Confidence: {'HIGH' if np.std(fold_map3_scores) < 0.01 else 'MEDIUM' if np.std(fold_map3_scores) < 0.02 else 'LOW'}")

In [15]:
%%time

print("Generating test predictions...")
test_predictions = MAP3Optimizer.generate_combined_predictions(
    test_cat_preds, test_misc_preds, categories, misconceptions
)

# Create submission
submission_data = []
for i, preds in enumerate(test_predictions):
    row_id = test.iloc[i]['row_id']
    pred_str = ' '.join(preds)
    submission_data.append({'row_id': row_id, 'Category:Misconception': pred_str})

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission.csv', index=False)

print(f"Submission created with {len(submission_df)} rows")
print("\nSample predictions:")
for i in range(min(3, len(test_predictions))):
    print(f"  Test {i}: {' '.join(test_predictions[i])}")

print("\nSubmission file:")
print(submission_df.head())

Generating test predictions...
Submission created with 3 rows

Sample predictions:
  Test 0: True_Correct:NA False_Neither:NA False_Misconception:Adding_across
  Test 1: False_Neither:NA True_Correct:NA False_Misconception:Incomplete
  Test 2: True_Neither:NA True_Correct:NA False_Misconception:Irrelevant

Submission file:
   row_id                             Category:Misconception
0   36696  True_Correct:NA False_Neither:NA False_Misconc...
1   36697  False_Neither:NA True_Correct:NA False_Misconc...
2   36698  True_Neither:NA True_Correct:NA False_Misconce...
CPU times: user 2.25 ms, sys: 1.06 ms, total: 3.3 ms
Wall time: 3.28 ms


## 7. Feature Analysis and Insights

In [16]:
print("=== Feature Analysis ===")

# Mathematical features analysis
print("\nTop Mathematical Features (by variance):")
math_variance = train_math_df.var().sort_values(ascending=False)
print(math_variance.head(10))

# Semantic features analysis
print("\nSemantic Feature Statistics:")
print(train_semantic_df.describe())

# Category distribution analysis
print("\nCategory Prediction Confidence:")
cat_confidence = np.max(oof_cat_preds, axis=1)
print(f"Mean: {cat_confidence.mean():.3f}, Std: {cat_confidence.std():.3f}")

print("\nMisconception Prediction Confidence:")
misc_confidence = np.max(oof_misc_preds, axis=1)
print(f"Mean: {misc_confidence.mean():.3f}, Std: {misc_confidence.std():.3f}")

# Error analysis by category
print("\nAccuracy by Category:")
for cat in categories:
    mask = train['Category'] == cat
    if mask.sum() > 0:
        cat_true = train.loc[mask, 'target_combined'].tolist()
        cat_pred = [oof_predictions[i][0] for i in range(len(oof_predictions)) if mask.iloc[i]]
        cat_acc = np.mean([t == p for t, p in zip(cat_true, cat_pred)])
        print(f"  {cat}: {cat_acc:.3f} (n={mask.sum()})")

=== Feature Analysis ===

Top Mathematical Features (by variance):
number_range         3.218397e+07
max_number           3.218391e+07
avg_number           8.131850e+05
text_length          5.906509e+03
word_count           2.267055e+02
min_number           1.867986e+02
number_count         1.522297e+01
operation_count      5.163082e+00
parentheses_count    4.087267e+00
sentence_count       3.208842e+00
dtype: float64

Semantic Feature Statistics:
       question_answer_similarity  question_explanation_similarity  \
count                36696.000000                     36696.000000   
mean                     0.144662                         0.059529   
std                      0.140718                         0.056860   
min                      0.000000                         0.000000   
25%                      0.000000                         0.000000   
50%                      0.100000                         0.052632   
75%                      0.181818                         

print("="*60)
print("=== ADVANCED BASELINE V2.0 SUMMARY ===")
print("="*60)

print(f"\n📊 PERFORMANCE METRICS (MAP@3):")
print(f"   Cross-Validation: {cv_map3:.6f} ± {np.std(fold_map3_scores):.6f}")
print(f"   Out-of-Fold: {overall_map3:.6f}")
print(f"   CV Stability: {np.std(fold_map3_scores)/np.mean(fold_map3_scores)*100:.2f}%")

print(f"\n🎯 COMPETITION TARGETS:")
improvement_vs_leader = cv_map3 - 0.841
improvement_vs_fork = cv_map3 - 0.852
status_leader = "✅ ACHIEVED" if improvement_vs_leader > 0 else "❌ MISSED"
status_fork = "✅ ACHIEVED" if improvement_vs_fork > 0 else "❌ MISSED"
print(f"   vs Current #1 (0.841): {improvement_vs_leader:+.6f} ({status_leader})")
print(f"   vs Fork baseline (0.852): {improvement_vs_fork:+.6f} ({status_fork})")

print(f"\n🔧 FEATURES IMPLEMENTED:")
print(f"   ✅ Mathematical Feature Engineering ({train_math_df.shape[1]} features)")
print(f"   ✅ Semantic Relationship Features ({train_semantic_df.shape[1]} features)")
print(f"   ✅ Enhanced Text Processing")
print(f"   ✅ MAP@3 Optimized Predictions")
print(f"   ✅ PROPER MAP@3 Cross-Validation ⭐")
print(f"   ✅ Robust Error Handling")
print(f"   ✅ LightGBM for Stability")

print(f"\n📈 NEXT IMPROVEMENT OPPORTUNITIES:")
if cv_map3 <= 0.841:
    print(f"   🔥 PRIORITY: Close gap of {0.841 - cv_map3:.6f} to beat #1")
    print(f"   🎯 Strategy 1: Advanced Transformer Models")
    print(f"   🎯 Strategy 3: Ensemble Methods")
    print(f"   🎯 Feature Engineering Iteration")
elif cv_map3 <= 0.852:
    print(f"   🎯 Goal: Beat fork baseline (gap: {0.852 - cv_map3:.6f})")
    print(f"   🔮 Strategy 1: Advanced Transformer Models")
    print(f"   🔮 Strategy 3: Sophisticated Ensemble")
else:
    print(f"   🚀 Excellent baseline! Focus on:")
    print(f"   🔮 Ensemble diversity")
    print(f"   🔮 Model stability")
    print(f"   🔮 Overfitting prevention")

print(f"\n📁 FILES CREATED:")
print(f"   📄 submission.csv (ready for Kaggle submission)")
print(f"   📊 Validation MAP@3: {cv_map3:.6f}")

print(f"\n🎯 VALIDATION CONFIDENCE:")
cv_std = np.std(fold_map3_scores)
if cv_std < 0.01:
    confidence = "🟢 HIGH"
elif cv_std < 0.02:
    confidence = "🟡 MEDIUM" 
else:
    confidence = "🔴 LOW"
print(f"   {confidence} (CV std: {cv_std:.6f})")

print(f"\n" + "="*60)
if cv_map3 > 0.841:
    print("🎉 READY FOR SUBMISSION - EXPECTED TO BEAT #1! 🚀")
else:
    print("📈 GOOD BASELINE - NEEDS FURTHER IMPROVEMENT")
print("="*60)

In [17]:
print("="*60)
print("=== ADVANCED BASELINE V2.0 SUMMARY ===")
print("="*60)

print(f"\n📊 PERFORMANCE METRICS:")
print(f"   MAP@3 Score: {overall_map3:.6f}")
print(f"   Top-1 Accuracy: {top1_acc:.6f}")
print(f"   CV Stability: {np.std(fold_scores):.6f}")

print(f"\n🎯 COMPETITION TARGETS:")
improvement_vs_leader = overall_map3 - 0.841
improvement_vs_fork = overall_map3 - 0.852
print(f"   vs Current #1 (0.841): {improvement_vs_leader:+.6f} ({'ACHIEVED' if improvement_vs_leader > 0 else 'MISSED'})")
print(f"   vs Fork baseline (0.852): {improvement_vs_fork:+.6f} ({'ACHIEVED' if improvement_vs_fork > 0 else 'MISSED'})")

print(f"\n🔧 FEATURES IMPLEMENTED:")
print(f"   ✅ Mathematical Feature Engineering ({train_math_df.shape[1]} features)")
print(f"   ✅ Semantic Relationship Features ({train_semantic_df.shape[1]} features)")
print(f"   ✅ Enhanced Text Processing")
print(f"   ✅ MAP@3 Optimized Predictions")
print(f"   ✅ Robust Error Handling")
print(f"   ✅ LightGBM for Stability")

print(f"\n📈 NEXT IMPROVEMENT OPPORTUNITIES:")
print(f"   🔮 Strategy 1: Advanced Transformer Models (DeBERTa/MathBERT)")
print(f"   🔮 Strategy 3: Sophisticated Ensemble Methods")
print(f"   🔮 Strategy 5: Data Augmentation & External Data")
print(f"   🔮 Hyperparameter Optimization")
print(f"   🔮 Feature Selection & Engineering")

print(f"\n📁 FILES CREATED:")
print(f"   📄 submission.csv (ready for Kaggle submission)")

print(f"\n" + "="*60)
print("Ready for submission to Kaggle! 🚀")
print("="*60)

=== ADVANCED BASELINE V2.0 SUMMARY ===

📊 PERFORMANCE METRICS:
   MAP@3 Score: 0.544446
   Top-1 Accuracy: 0.455717
   CV Stability: 0.005700

🎯 COMPETITION TARGETS:
   vs Current #1 (0.841): -0.296554 (MISSED)
   vs Fork baseline (0.852): -0.307554 (MISSED)

🔧 FEATURES IMPLEMENTED:
   ✅ Mathematical Feature Engineering (32 features)
   ✅ Semantic Relationship Features (6 features)
   ✅ Enhanced Text Processing
   ✅ MAP@3 Optimized Predictions
   ✅ Robust Error Handling
   ✅ LightGBM for Stability

📈 NEXT IMPROVEMENT OPPORTUNITIES:
   🔮 Strategy 1: Advanced Transformer Models (DeBERTa/MathBERT)
   🔮 Strategy 3: Sophisticated Ensemble Methods
   🔮 Strategy 5: Data Augmentation & External Data
   🔮 Hyperparameter Optimization
   🔮 Feature Selection & Engineering

📁 FILES CREATED:
   📄 submission.csv (ready for Kaggle submission)

Ready for submission to Kaggle! 🚀
